In [9]:
import argparse
import os
import numpy as np
import math
import random 

import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision.models as models
from PIL import Image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import os

In [10]:

n_epochs = 10
batch_size = 64
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 8
latent_dim = 100
n_classes = 2
img_size = 224
channels = 3
sample_interval = 50

img_shape = (3, 224, 224)

In [11]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(1124, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels, featureExtractor):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((featureExtractor.extract_feature(labels), noise), -1).to(device)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(151552, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels, featureExtractor):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), featureExtractor.extract_feature(labels)), -1)
        validity = self.model(d_in)
        return validity

In [12]:
class feature_extractor():
    def __init__(self):
        self.model = models.googlenet(pretrained=True)
        self.model.fc = nn.Identity()
        self.model.eval()

    def extract_feature(self, labels):
        device = next(self.model.parameters()).device 
        features = torch.zeros(len(labels), 1024).to(device)
        for i, label in enumerate(labels):
            if label == 0:  # cat
                input_image = Image.open("/kaggle/input/dog-vs-cat/dogs_vs_cats/test/cats/cat.10.jpg")
            else:  # dog
                input_image = Image.open("/kaggle/input/dog-vs-cat/dogs_vs_cats/test/dogs/dog.100.jpg")
            preprocess = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor()
            ])
            input_tensor = preprocess(input_image)
            input_tensor = input_tensor.reshape(1, 3, 224, 224).to(device)
            
            with torch.no_grad():
                features[i] = self.model(input_tensor).squeeze()

        return features

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize generator and discriminator
featureExtractor = feature_extractor()  # Create an instance of the feature_extractor class
featureExtractor.model.to(device)  # Move the feature extractor model to the device
generator = Generator().to(device)
discriminator = Discriminator().to(device)
adversarial_loss = torch.nn.MSELoss().to(device)

optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))


In [14]:
dataloader = torch.utils.data.DataLoader(
    datasets.ImageFolder(
        "/kaggle/input/dog-vs-cat/dogs_vs_cats/train",
        transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ]),
    ),
    batch_size=64,
    shuffle=True,
)

In [15]:
os.makedirs("images", exist_ok=True)

def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = torch.tensor(np.random.normal(0, 1, (n_row ** 2, latent_dim)), dtype=torch.float32).cuda()
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = torch.tensor(labels, dtype=torch.int64).cuda()
    gen_imgs = generator(z, labels, featureExtractor)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


## modified version

In [16]:
for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = torch.ones((batch_size, 1), requires_grad=False, device=device, dtype=torch.float32)
        fake = torch.zeros((batch_size, 1), requires_grad=False, device=device, dtype=torch.float32)

        # Configure input
        real_imgs = imgs.to(device)
        labels = labels.to(device)

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = torch.FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))).to(device)
        gen_labels = torch.LongTensor(np.random.randint(0, n_classes, batch_size)).to(device)

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels, featureExtractor).to(device)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels, featureExtractor)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels, featureExtractor)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels, featureExtractor)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        if i % 50 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )
        

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)
            
            
    save_image(gen_imgs, f"generated_image_{epoch + 1}.png", normalize=True)

torch.save(generator.state_dict(), "/kaggle/working/generator.pt")
torch.save(discriminator.state_dict(), "/kaggle/working/discriminator.pt")

[Epoch 0/10] [Batch 0/313] [D loss: 0.476565] [G loss: 0.943101]
[Epoch 0/10] [Batch 50/313] [D loss: 0.285714] [G loss: 0.748921]
[Epoch 0/10] [Batch 100/313] [D loss: 0.242396] [G loss: 0.613848]
[Epoch 0/10] [Batch 150/313] [D loss: 0.154550] [G loss: 0.728655]
[Epoch 0/10] [Batch 200/313] [D loss: 0.107355] [G loss: 0.909253]
[Epoch 0/10] [Batch 250/313] [D loss: 6.234743] [G loss: 1.119548]
[Epoch 0/10] [Batch 300/313] [D loss: 0.196294] [G loss: 0.837846]
[Epoch 1/10] [Batch 0/313] [D loss: 0.213740] [G loss: 0.638044]
[Epoch 1/10] [Batch 50/313] [D loss: 0.170416] [G loss: 0.919703]
[Epoch 1/10] [Batch 100/313] [D loss: 0.181143] [G loss: 0.676754]
[Epoch 1/10] [Batch 150/313] [D loss: 0.105669] [G loss: 0.838436]
[Epoch 1/10] [Batch 200/313] [D loss: 0.089800] [G loss: 0.738095]
[Epoch 1/10] [Batch 250/313] [D loss: 0.106805] [G loss: 0.695514]
[Epoch 1/10] [Batch 300/313] [D loss: 0.113409] [G loss: 0.703705]
[Epoch 2/10] [Batch 0/313] [D loss: 0.121076] [G loss: 0.624170]
[Ep

In [65]:
import torch
from torchvision.transforms import ToTensor, transforms
from PIL import Image

# Load the generator and discriminator models
generator.load_state_dict(torch.load("/kaggle/working/generator.pt"))
discriminator.load_state_dict(torch.load("/kaggle/working/discriminator.pt"))

# Path to the input image
input_image_path = "/kaggle/input/dog-vs-cat/dogs_vs_cats/train/cats/cat.0.jpg"

# Load and preprocess the input image
input_image = Image.open(input_image_path)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
input_tensor = preprocess(input_image).unsqueeze(0).to(device)

# Generate the output image
with torch.no_grad():
    z = torch.FloatTensor(1, latent_dim).normal_().to(device)  # Random noise generation
    gen_labels = torch.LongTensor(1).random_(0, n_classes).to(device)  # Random label generation

    # Concatenate label embedding and image to produce input
    feature_embedding = featureExtractor.extract_feature(input_tensor,0).view(1, -1)
    gen_input = torch.cat((feature_embedding, z), dim=1).to(device)
    generated_image = generator(gen_input)
    generated_image = (generated_image + 1) / 2

# Show the input and generated images
input_image.show()
generated_image = generated_image.squeeze().permute(1, 2, 0).cpu().numpy()
generated_image_pil = Image.fromarray((generated_image * 255).astype("uint8"))
generated_image_pil.show()

# Save the generated image
output_path = "/kaggle/working/generated_image.png"
generated_image_pil.save(output_path)


TypeError: feature_extractor.extract_feature() takes 2 positional arguments but 3 were given

In [36]:
g_loss.item()

0.8097712993621826